In [50]:
import pandas as pd
import numpy as np
import altair as alt

In [51]:
df = pd.read_csv('mixed_model_results_yearly_stats_21_24_aug_24.csv')
pos_info = pd.read_csv('career_stats_with_predicted_positions_aug_24.csv')
df['position'] = df.apply(lambda row: pos_info[pos_info['name'] == row['name']]['predicted_position'].values[0] if not pos_info[pos_info['name'] == row['name']].empty else '', axis=1)
# result_df['position'] = df.apply(lambda row: pos_info[pos_info['name'] == row['name']]['position'].values[0] if not pos_info[pos_info['name'] == row['name']].empty else '', axis=1)
df['position'] = df.apply(lambda row: pos_info[pos_info['name'] == row['name']]['position'].iloc[0] if (len(pos_info[pos_info['name'] == row['name']]['position']) > 0 and pos_info[pos_info['name'] == row['name']]['position'].iloc[0] in ['Cutter', 'Handler', 'Defender']) else row['position'], axis=1)
df
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'playerID', 'name', 'gamesPlayed',
       'scores', 'assists', 'goals', 'plusMinus', 'completions',
       'completionPercentage', 'hockeyAssists', 'throwaways', 'stalls',
       'drops', 'blocks', 'callahans', 'pulls', 'teams', 'year',
       'pointsPlayed', 'oPointsPlayed', 'dPointsPlayed', 'minutesPlayed',
       'possessions', 'oEfficiency', 'yardsTotal', 'yardsThrown',
       'yardsReceived', 'hucksCompleted', 'huckPercentage', 'oeff_rating',
       'goal_rating', 'assist_rating', 'block_rating', 'composite_rating',
       'position'],
      dtype='object')

In [52]:
df = df[df['year']>=2021]
df = df[df['composite_rating'].notna()]
df.shape

(2919, 37)

In [53]:
results = df.groupby('name').agg({'position': 'first', 'gamesPlayed':'sum', 'year':'count',
                                  'oeff_rating': 'first', 'goal_rating':'first',
                                  'assist_rating':'first', 'block_rating': 'first',
                                  'composite_rating': 'first'})
results.shape

(1343, 8)

In [54]:
qualifiers = results[results['year']>=3]
qualifiers = qualifiers[qualifiers['gamesPlayed'] >= 30]
qualifiers.reset_index(inplace=True)
qualifiers.rename(columns = {'gamesPlayed': 'games', 'year': 'years', 'oeff_rating': 'oeff rating',
                          'goal_rating': 'goal rating', 'assist_rating': 'assist rating',
                          'block_rating': 'block rating', 'composite_rating': 'composite rating'}, inplace = True)
qualifiers = qualifiers.round(2)
qualifiers.sort_values('composite rating', ascending=False).head(10)

,name,position,games,years,oeff rating,goal rating,assist rating,block rating,composite rating
212,Sean McDougall,Cutter,51,4,0.71,3.13,1.64,3.04,2.13
205,Ryan Osgar,Handler,42,3,0.64,2.04,6.71,-0.98,2.10
189,Pawel Janas,Handler,53,4,2.70,-1.21,5.25,-0.98,1.44
200,Rocco Linehan,Handler,42,4,1.34,1.21,1.09,2.12,1.44
17,Ben Jagt,Cutter,55,4,0.01,3.37,1.02,1.14,1.39
123,Jordan Kerr,Handler,52,4,-0.77,1.33,3.91,1.02,1.37
2,Alex Atkins,Handler,30,3,-0.40,1.78,0.96,2.69,1.26
106,James Pollard,Cutter,47,4,-0.68,1.64,1.41,2.65,1.25
88,Jace Bruner,Defender,35,4,1.39,1.20,-0.51,2.89,1.24
21,Ben Lewis,Defender,47,4,0.38,3.79,-0.04,0.84,1.24


In [55]:
df1 = qualifiers[['name', 'position', 'composite rating']]

In [56]:
df = pd.read_csv('final_yearly_df_aug_24.csv')
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'playerID', 'name', 'gamesPlayed',
       'scores', 'assists', 'goals', 'plusMinus', 'completions',
       'completionPercentage', 'hockeyAssists', 'throwaways', 'stalls',
       'drops', 'blocks', 'callahans', 'pulls', 'teams', 'year',
       'pointsPlayed', 'oPointsPlayed', 'dPointsPlayed', 'minutesPlayed',
       'possessions', 'oEfficiency', 'yardsTotal', 'yardsThrown',
       'yardsReceived', 'hucksCompleted', 'huckPercentage', 'player_id',
       'oscorerate', 'dscorerate', 'o_point_on_off_rating',
       'd_point_on_off_rating', 'total_on_off_rating', 'position'],
      dtype='object')

In [57]:
df['position'] = df.apply(lambda row: pos_info[pos_info['name'] == row['name']]['predicted_position'].values[0] if not pos_info[pos_info['name'] == row['name']].empty else '', axis=1)
# result_df['position'] = df.apply(lambda row: pos_info[pos_info['name'] == row['name']]['position'].values[0] if not pos_info[pos_info['name'] == row['name']].empty else '', axis=1)
df['position'] = df.apply(lambda row: pos_info[pos_info['name'] == row['name']]['position'].iloc[0] if (len(pos_info[pos_info['name'] == row['name']]['position']) > 0 and pos_info[pos_info['name'] == row['name']]['position'].iloc[0] in ['Cutter', 'Handler', 'Defender']) else row['position'], axis=1)


In [58]:
df['o point on off rating'] = df['o_point_on_off_rating']*df['oPointsPlayed']
df['d point on off rating'] = df['d_point_on_off_rating']*df['dPointsPlayed']

results = df.groupby('name').agg({'position': 'first', 'gamesPlayed': 'sum', 'pointsPlayed': 'sum', 'year': 'count',
                                  'oPointsPlayed': 'sum', 'dPointsPlayed': 'sum',
                                  'o point on off rating': 'sum', 'd point on off rating' : 'sum'})

In [59]:
results['total on off rating'] = (results['o point on off rating'] + results['d point on off rating'])/ results['pointsPlayed']
results['o point on off rating'] = results['o point on off rating'] / results['oPointsPlayed']
results['d point on off rating'] = results['d point on off rating'] / results['dPointsPlayed']

In [60]:
qualifiers = results[results['year']>=3]
qualifiers = qualifiers[qualifiers['gamesPlayed'] >= 30]
qualifiers.reset_index(inplace=True)
qualifiers.rename(columns = {'gamesPlayed': 'games', 'year': 'years', 'o point on off rating': 'offensive on off rating',
                             'd point on off rating' : 'defensive on off rating'}, inplace = True)
qualifiers = qualifiers.round(2)
qualifiers = qualifiers[['name', 'position', 'games', 'years', 'offensive on off rating', 'defensive on off rating', 'total on off rating']]
qualifiers.sort_values('total on off rating', ascending=False).head(10)

,name,position,games,years,offensive on off rating,defensive on off rating,total on off rating
94,Jack Williams,Handler,55,4,25.82,-0.06,23.48
14,Austin Taylor,Cutter,50,4,20.13,8.18,19.69
205,Ryan Osgar,Handler,42,3,19.73,-1.62,19.56
189,Pawel Janas,Handler,53,4,18.55,1.85,18.43
25,Bobby Ley,Handler,43,4,17.87,4.92,17.73
98,Jacob Miller,Handler,52,4,19.72,7.96,17.18
66,Elliott Chartock,Handler,49,4,17.31,12.82,17.00
211,Sean Connole,Handler,40,3,17.52,8.18,16.98
145,Luc Comire,Handler,40,4,17.89,6.93,16.95
119,John Lithio,Cutter,52,4,17.61,-11.74,16.29


In [46]:
qualifiers.sort_values('total on off rating', ascending=False, inplace= True)
qualifiers.reset_index(inplace=True)

In [47]:
qualifiers.sort_values('defensive on off rating', ascending=False).head(10)

,index,name,position,games,years,offensive on off rating,defensive on off rating,total on off rating
43,239,Tyler Monroe,Handler,40,3,10.55,25.36,10.73
72,122,Jonny Malks,Handler,51,4,6.73,21.01,6.95
51,112,Jeff Wodatch,Cutter,44,4,9.65,20.15,9.76
12,149,Luke Yorgason,Handler,40,3,15.70,20.15,15.81
48,128,Josh Klane,Handler,48,4,9.89,15.33,10.14
95,220,Spencer Lofink,Cutter,31,4,1.31,14.26,5.29
50,120,Jonathan Mast,Handler,31,3,9.83,13.71,9.85
90,172,Michael Jordan,Cutter,36,4,5.77,12.92,6.12
6,66,Elliott Chartock,Handler,49,4,17.31,12.82,17.00
32,77,Everest Shapiro,Cutter,32,3,11.45,11.62,11.45


In [48]:
qualifiers[qualifiers['position']=='Defender'].sort_values('total on off rating', ascending=False).head(10)

,index,name,position,games,years,offensive on off rating,defensive on off rating,total on off rating
61,88,Jace Bruner,Defender,35,4,-16.15,10.25,8.10
76,12,Antoine Davis,Defender,51,4,5.17,7.56,6.85
78,111,Jeff Babbitt,Defender,58,4,11.53,-1.24,6.78
80,43,Charlie Furse,Defender,35,3,-31.50,7.71,6.62
91,21,Ben Lewis,Defender,47,4,6.59,5.40,6.02
92,32,Brett Hulsmeyer,Defender,46,4,12.75,-0.21,5.89
105,221,Stefan Samu,Defender,35,4,1.94,5.66,4.47
108,51,Cole Jurek,Defender,45,4,6.08,-3.41,4.29
119,52,Colin Berry,Defender,43,4,5.95,1.33,3.75
121,68,Eric Brodbeck,Defender,52,4,6.12,1.39,3.73


In [44]:
qualifiers[qualifiers['position']=='Cutter'].sort_values('total on off rating', ascending=False).head(10)

,index,name,position,games,years,offensive on off rating,defensive on off rating,total on off rating
1,14,Austin Taylor,Cutter,50,4,20.13,8.18,19.69
9,119,John Lithio,Cutter,52,4,17.61,-11.74,16.29
10,141,Kyle Henke,Cutter,51,4,18.92,-1.20,16.18
13,116,Jimmy Towle,Cutter,43,4,16.06,0.64,15.64
14,84,Henry Fisher,Cutter,40,4,18.30,9.81,14.88
16,213,Sean Mott,Cutter,49,4,15.43,-0.62,14.68
19,235,Travis Dunn,Cutter,46,4,14.90,6.84,14.34
23,173,Michael Kiyoi,Cutter,46,4,13.93,3.99,13.89
24,36,CJ Colicchio,Cutter,31,3,16.08,-4.64,13.72
25,96,Jacob Fairfax,Cutter,56,4,15.04,-3.45,13.50


In [61]:
df1 = df1.merge(qualifiers[['name', 'total on off rating']], on = 'name', how = 'left')

In [63]:
all_audl_teams_23 = ['Jeff Babbitt', 'Brett Hulsmeyer', 'Ben Jagt', 'Pawel Janas', 'Jordan Kerr', 'Sean McDougall',
                 'Ryan Osgar', 'Christian Boxley', 'Quinn Finer', 'Kyle Henke', 'John Randolph', 'Max Sheppard',
                 'Evan Swiatek', 'Jack Williams', 'Antoine Davis', 'Travis Dunn', 'Jacob Fairfax', 'Mac Hecht',
                 'Jonny Malks', 'Garrett Martin', 'Jacob Miller']

all_audl_teams_22 = ['Ben Jagt', 'Pawel Janas', 'Jordan Kerr', 'Rowan McDonnell', 'Jonathan Nethercutt',
                    'Ryan Osgar', 'Jack Williams', 'Jeff Babbitt', 'Travis Dunn', 'Quinn Finer', 'Kyle Henke',
                    'Leandro Marx', 'Cody Spicer', 'Eric Taylor']

all_audl_teams_21 = ['Travis Dunn', 'Ben Jagt', 'Pawel Janas', 'Ryan Osgar', 'Austin Taylor',
                    'Jack Williams', 'Jeff Babbitt', 'Allan Laviolette', 'Jonny Malks', 'Greg Martin',
                    'Steven Milardovich', 'Benjamin Sadok', 'Max Sheppard']

mvps_23 = ['Jeff Babbitt']
mvps_22 = ['Ryan Osgar']
mvps_21 = ['Ben Jagt']



df1['All AUDL Team'] = df1['name'].apply(lambda x: x in ((all_audl_teams_21+ all_audl_teams_22+ all_audl_teams_23)))


sum(df1['All AUDL Team'])


23

In [65]:
alt.Chart(df1).mark_circle(lineBreak = r'\n').encode(
    alt.X('total on off rating', title='Total On-Off Rating'), 
    alt.Y('composite rating', title='Composite Rating'),  
    color = 'All AUDL Team:N',
    tooltip='name:N'
).properties(
    title=['Total on-off Rating vs. Composite Rating','ALL-AUDL Teams Highlighted (2021-2023)']
).configure_axis(
    titleFontSize=18
).configure_title(
    fontSize=20  )

alt.Chart(...)